### decorator

In [3]:
import functools
import time
from loguru import logger
from typing import Callable, Any, Optional


# timer class
class Timer:
    def __init__(self, timeout: int = 5):
        self.timeout = timeout
        self.start_time = time.time()

    def is_timeout(self) -> bool:
        return (time.time() - self.start_time) > self.timeout


class MaxRetriesExceeded(Exception):
    pass


def retry_till_timeout(
    delay=1,
    timeout=10,
):
    """retry until time out"""

    def decorate(func):
        @functools.wraps(func)
        def new_func(*args, **kwargs):
            timer = Timer(timeout)
            while True:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if timer.is_timeout():
                        logger.warning(
                            f"{func.__name__} failed with error: {e}, reached timeout"
                        )
                        raise MaxRetriesExceeded(
                            f"Timeout {timeout} sec reached for {func.__name__}"
                        )
                    else:
                        logger.warning(
                            f"{func.__name__} failed with error: {e}, will retry in {delay} seconds..."
                        )
                        time.sleep(delay)

        return new_func

    return decorate

In [4]:
import requests


# Example usage
@retry_till_timeout(delay=1, timeout=10)
def make_request(url):
    """
    Makes a request to the given URL.
    """
    response = requests.get(url)
    response.raise_for_status()
    return response


test_url = "https://httpbin.org/status/429"  # Simulates rate-limiting
try:
    response = make_request(test_url)
    print("Response:", response.text)
except MaxRetriesExceeded as e:
    logger.error(e)

2025-04-23 00:26:48.566 | WARNING  | __main__:new_func:37 - make_request failed with error: 503 Server Error: Service Temporarily Unavailable for url: https://httpbin.org/status/429, will retry in 1 seconds...
2025-04-23 00:26:50.616 | WARNING  | __main__:new_func:37 - make_request failed with error: 503 Server Error: Service Temporarily Unavailable for url: https://httpbin.org/status/429, will retry in 1 seconds...
2025-04-23 00:26:52.675 | WARNING  | __main__:new_func:37 - make_request failed with error: 503 Server Error: Service Temporarily Unavailable for url: https://httpbin.org/status/429, will retry in 1 seconds...
2025-04-23 00:26:54.744 | WARNING  | __main__:new_func:37 - make_request failed with error: 503 Server Error: Service Temporarily Unavailable for url: https://httpbin.org/status/429, will retry in 1 seconds...
2025-04-23 00:26:56.896 | WARNING  | __main__:new_func:37 - make_request failed with error: 503 Server Error: Service Temporarily Unavailable for url: https://h

### timer

In [ ]:
import time


# timer class
class Timer:
    def __init__(self, timeout: int = 5):
        self.timeout = timeout
        self.start_time = time.time()

    def reset(self):
        self.start_time = time.time()

    def extend(self, timeout: int):
        self.timeout += timeout

    def is_timeout(self) -> bool:
        return (time.time() - self.start_time) > self.timeout

    def elapsed_time(self) -> float:
        return round(time.time() - self.start_time, 2)

In [ ]:
# test
timer = Timer(3)
time.sleep(2)
assert timer.is_timeout() == False, "Timer should not be timed out yet"
time.sleep(2)
assert timer.is_timeout(), "Timer should be timed out now"

timer.extend(2)
assert timer.is_timeout() == False, "Timer should not be timed out yet"

timer.reset()
time.sleep(1)
assert timer.is_timeout() == False, "Timer should not be timed out yet"